# RCTデータで検証

## RCTを行ったデータの準備

**Rコード**

```R
# ライブラリーimport
library(tidyverse)
library(broom)

# データフレーム読み込み
email_data <- read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

# 男性のみに限定
male_df <- email_data %>%
    filter(segment != "Womens E-Mail") %>%
    mutate(
        treatment = if_else(segment == "Mens E-Mail", 1, 0)
    )
```

**データ説明**

|変数名|説明|
|:-------|:------|
|recency|最後の購入からの経過月数|
|history_segment|昨年の購入額の階層|
|history|昨年の購入額|
|mens|昨年に男物の商品を購入しているか|
|womens|昨年に女物の商品を購入しているか|
|zipcode|zipcodeをもとに地区を分類したもの|
|newbie|過去12ヶ月以内に新しくユーザになったか|
|channel|昨年においてどのチャンネルから購入したか|
|segment|どのメールが配信されたか|
|visit|メールが配信されてから2週間以内にサイトへ来訪したか|
|conversion|メールが配信されてから2週間以内に購入したか|
|spend|購入した際の購入額|

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, uniform
import statsmodels.api as sm

In [2]:
df_email = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

In [3]:
df_male = df_email.query("segment != 'Womens E-Mail'").copy()
df_male["treatment"] = np.where(df_male["segment"] == "Mens E-Mail", 1, 0)
df_male.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0.0,0


## RCTデータの集計と有意差検定

**Rコード**
```R
# 集計による比較
summary_by_segment <- male_df %>%
    group_by(treatment) %>%
    summarise(
        conversion_rate = mean(conversion),
        spend_mean = mean(spend),
        count = n(),
        .groups = "drop"
    )

# 介入群の購買データ
mens_mail <- male_df %>%
    filter(treatment == 1) %>%
    pull(spend)

# コントロールの購買データ
no_mail <- male_df %>%
    filter(treatment == 0) %>%
    pull(spend)

# 2群の平均についてt検定
rct_ttest <- t.test(mens_mail, no_mail, var.equal = FALSE)
rct_ttest
```

In [4]:
df_summary_by_segment = (df_male
                         .groupby("treatment")
                         .agg({"conversion": "mean", "spend": "mean", "channel": "size"})
                         .reset_index()
                         .rename(columns={"channel": "count"})
                        )
df_summary_by_segment

,treatment,conversion,spend,count
0,0,0.005726,0.652789,21306
1,1,0.012531,1.422617,21307


In [5]:
mens_mail = df_male.query("treatment == 1")["spend"]
no_mail = df_male.query("treatment == 0")["spend"]

rct_ttest = ttest_ind(mens_mail, no_mail, equal_var=False)
rct_ttest

Ttest_indResult(statistic=5.300140358411668, pvalue=1.1638149682254859e-07)

# バイアスのあるデータで検証

## バイアスのあるデータの準備

**Rコード**
```R
set.seed(1)

# 条件に反応するサンプルの量を半分にする
obs_rate_c <- 0.5
obs_rate_t <- 0.5

# バイアスのあるデータの作成
biased_data <- male_df %>%
    mutate(
        obs_rate_c = if_else(
            (history > 300) | (recency < 6) | (channel == "Multichannel"),
            obs_rate_c,
            1
        ),
        obs_rate_t = if_else(
            (history > 300) | (recency < 6) | (channel == "Multichannel"),
            1,
            obs_rate_t
        ),
        random_number = runif(n = NROW(male_df))
    ) %>%
    filter(
        (treatment == 0 & random_number < obs_rate_c) |
        (treatment == 1 & random_number < obs_rate_t)
    )
```

In [6]:
obs_rate_c = 0.5
obs_rate_t = 0.5

df_biased_email = (df_male
                   .assign(
                       obs_rate_c=np.where(
                           (df_male["history"] > 300) | (df_male["recency"] < 6) | (df_male["channel"] == "Multichannel"),
                           obs_rate_c,
                           1
                       )
                   )
                   .assign(
                       obs_rate_t=np.where(
                           (df_male["history"] > 300) | (df_male["recency"] < 6) | (df_male["channel"] == "Multichannel"),
                           obs_rate_t,
                           1
                       )
                   )
                   .assign(random_number=uniform.rvs(size=len(df_male), random_state=46))
                   .query("(treatment == 0 & random_number < obs_rate_c) | (treatment == 1 & random_number < obs_rate_t)")
)

## バイアスのあるデータの集計と有意差検定

**Rコード**
```R
summary_by_segment_biased <- biased_data %>%
    group_by(treatment) %>%
    summarise(
        conversion_rate = mean(conversion),
        spend_mean = mean(spend),
        count = n(),
        .groups = "drop"
    )

# 介入群の購買データ
mens_mail_biased <- biased_data %>%
    filter(treatment == 1) %>%
    pull(spend)

# コントロール群の購買データ
no_mail_biased <- biased_data %>%
    filter(treatment == 0) %>%
    pull(spend)

# 2群の平均の差についてt検定
rct_ttest_biased <- t.test(mens_mail_biased, no_mail_biased, var.equal = F)
rct_ttest_biased
```

In [7]:
df_summary_by_segment_biased = (df_biased_email
                        .groupby("treatment")
                        .agg({"conversion": "mean", "spend": "mean", "channel": "size"})
                        .reset_index()
                        .rename(columns={"channel": "count"})
                        )
df_summary_by_segment_biased

,treatment,conversion,spend,count
0,0,0.005492,0.597426,14749
1,1,0.012376,1.445769,14867


In [8]:
mens_mail_biased = df_biased_email.query("treatment == 1")["spend"]
no_mail_biased = df_biased_email.query("treatment == 0")["spend"]

rct_ttest_biased = ttest_ind(mens_mail_biased, no_mail_biased, equal_var=False)
rct_ttest_biased

Ttest_indResult(statistic=4.8859950100455345, pvalue=1.0356835645899504e-06)

# 回帰分析でバイアスのあるデータの検証

## バイアスのあるデータで回帰分析

目的変数にユーザの購入学額(spend)をとり，介入変数(treatment)にメールの有無，そして共変量として過去の購入額(history)を使ったモデル

$Spend_i = \beta_0 + \beta_{treatment}treatment_i + \beta_{history}history_i$

**Rコード**
```R
biased_reg <- lm(data = biased_data, formula = spend ~ treatment + history)

biased_reg_coef <- tidy(biased_reg)
biased_reg_coef
```

In [9]:
X = sm.add_constant(df_biased_email[["treatment", "history"]])
y = df_biased_email["spend"]

model = sm.OLS(y, X)
biased_reg = model.fit()
print(biased_reg.summary())

                            OLS Regression Results                            
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     19.59
Date:                Thu, 18 Aug 2022   Prob (F-statistic):           3.16e-09
Time:                        02:42:22   Log-Likelihood:            -1.2215e+05
No. Observations:               29616   AIC:                         2.443e+05
Df Residuals:                   29613   BIC:                         2.443e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2974      0.145      2.050      0.0

## 効果検証のための回帰分析で行わないこと

効果検証のための回帰分析では，$\beta_{treatment}$以外の推定結果には興味がなく，それらのパラメータが本当の効果を表すようになる努力も行わないため，  
介入効果を示すパラメータ以外については無視することになる．有意差検定も同様に，興味のあるパラメータ以外の検定結果は一切解釈しない．  
分析の目的上，共変量のパラメータに関して興味がないため．そのパラメータが0でもそうでなくてもどちらでも良いということになる．

回帰分析によって得られたモデルは条件付き期待値を表すだけでなく，Yに対して誤差が最小になる式でもある．  

$E[Spend|history,treatment]$

$= \widehat{Spend}$

$= \hat{\beta_0} + \hat{\beta}treatment + \hat{\beta}history$

$= 0.3 + 0.85treatment + 0.002history$

この式にtreatmentとhistoryの値を代入すれば，spendに対する予測値である$\widehat{Spend_i}$を手に入れることができるため，  
回帰分析はまだ見ぬサンプルに対する予測に用いられることもある．

# 回帰分析におけるバイアス

## RCTを行ったデータでの単回帰

**Rコード**
```R
rct_reg <- lm(data = male_df, formula = spend ~ treatment)
rct_reg_coef <- summary(rct_reg) %>% tidy()
```

In [10]:
X = sm.add_constant(df_male[["treatment", "history"]])
y = df_male["spend"]

model = sm.OLS(y, X)
rct_reg = model.fit()
print(rct_reg.summary())

                            OLS Regression Results                            
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     23.30
Date:                Thu, 18 Aug 2022   Prob (F-statistic):           7.69e-11
Time:                        02:42:22   Log-Likelihood:            -1.7583e+05
No. Observations:               42613   AIC:                         3.517e+05
Df Residuals:                   42610   BIC:                         3.517e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3596      0.123      2.917      0.0

RCTを行っているデータはRCTの分析結果と同じになっている

## バイアスのあるデータでの単回帰

**Rコード**
```R
nonrct_reg <- lm(data = male_df, formula = spend ~ treatment)
nonrct_reg_coef <- summary(rct_reg) %>% tidy()
```

In [11]:
X = sm.add_constant(df_biased_email[["treatment"]])
y = df_biased_email["spend"]

model = sm.OLS(y, X)
nonrct_reg = model.fit()
print(nonrct_reg.summary())

                            OLS Regression Results                            
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     23.78
Date:                Thu, 18 Aug 2022   Prob (F-statistic):           1.09e-06
Time:                        02:42:22   Log-Likelihood:            -1.2216e+05
No. Observations:               29616   AIC:                         2.443e+05
Df Residuals:                   29614   BIC:                         2.443e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5974      0.123      4.847      0.0

バイアスのあるデータでは$\beta_{treatment}$の値が大きくなり，過剰に効果が見積もられている．

## バイアスのあるデータでの重回帰

**Rコード**
```R
nonrct_mreg <- lm(
    data = male_df,
    formula = spend ~ treatment + recency + channel + history
)
nonrct_mreg_coef <- summary(rct_reg) %>% tidy()
```

In [12]:
X = sm.add_constant(pd.get_dummies(df_biased_email[["treatment", "recency", "channel", "history"]], drop_first=True))
y = df_biased_email["spend"]

model = sm.OLS(y, X)
nonrct_mreg = model.fit()
print(nonrct_mreg.summary())

                            OLS Regression Results                            
Dep. Variable:                  spend   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     9.436
Date:                Thu, 18 Aug 2022   Prob (F-statistic):           5.31e-09
Time:                        02:42:22   Log-Likelihood:            -1.2215e+05
No. Observations:               29616   AIC:                         2.443e+05
Df Residuals:                   29610   BIC:                         2.444e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.2478      0.419      0.592

$\beta_{treatment}$はRCTにおける推定値に近づいている．